In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
import pickle
import numpy as np
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))
from groq import Groq
from src.llm_engine.gemini_agent import Agent


In [2]:
load_dotenv()

True

In [3]:
# client = Groq(
#     api_key= os.getenv("GROQ_API_KEY"),
# )

client = Groq(api_key= os.getenv("GROQ_API_KEY"))

In [4]:
df = pd.read_csv(r"data/GridPuzzle_processed.csv")

df.head()

,key,id,question,answer,Mistral-7b,Llama-13b,gemini-pro,gpt-4-turbo,Claude-3,actual_question,Clues,categories,number_of_clues,number_categories,category_names
0,341.json,802,Nicholas and Edith are having a small dinner p...,1984 | Annata Branco | gewurztraminer\n1988 | ...,| 1984 | Annata Branco | gewurztraminer |\n |-...,Let's solve the puzzle step by step.\n \n Step...,"Step-by-step solution:\n 1. From clue 6, we kn...","Step-by-step solution:\n \n 1. From clue 6, we...","Step-by-step solution:\n \n 1. From clue 6, we...",Nicholas and Edith are having a small dinner p...,Clues:\n1. The Ece Suss was bottled sometime a...,"vintages : 1984, 1988, 1992, 1996.\nwines : An...",6,3,vintages wines types
1,341.json,726,The world-famous Punta Santiago Rose Gardening...,first | bon silene | Renee\nsecond | perle d'o...,| first | calocarpa | Hannah\n |---------|----...,Let's break down the clues step by step:\n \n ...,"Step-by-step solution:\n 1. From clue 1, we kn...","Step-by-step solution:\n \n 1. From clue 3, we...","Step-by-step solution:\n \n 1. From clue 1, we...",The world-famous Punta Santiago Rose Gardening...,Clues:\n1. The rose that won fourth place was ...,"placements : first, second, third, fourth.\nro...",5,3,placements roses gardeners
2,341.json,16,A standardized test has recently been develope...,55% | Archimedes | CIT\n62% | Galen | UCLA\n69...,| 55% | Zeno | CIT |\n |------|------|-----|\n...,Let's break down the clues and solve the puzzl...,"Step-by-step solution:\n 1. From clue 2, we kn...","Step-by-step solution:\n \n 1. From clue 4, Pr...","Step-by-step solution:\n \n 1. From clue 4, we...",A standardized test has recently been develope...,Clues:\n1. Zeno finished with a score that was...,"scores : 55%, 62%, 69%, 76%.\na.i. : Archimede...",5,2,scores institutions
3,341.json,698,Gil Glabbers has lived on the coast of Guernse...,1962 | Tracy | Cherbourg\n1969 | Felipe | Bour...,| 1962 | Felipe (Bournemouth) |\n | 1969 | Kar...,Let's break down the clues step by step:\n \n ...,"Step-by-step solution:\n \n 1. From clue 2, we...","Step-by-step solution:\n \n 1. From Clue 2, Tr...","Step-by-step solution:\n \n 1. From clue 2, we...",Gil Glabbers has lived on the coast of Guernse...,Clues:\n1. The message from Plymouth was sent ...,"year sent : 1962, 1969, 1976, 1983.\nwriters :...",5,3,sent writers origins
4,341.json,444,Jamie never misses an opportunity to watch the...,2 | April 22 | Eastbrook\n9 | April 10 | Islet...,| Dates | Locations |\n |---|---|\n | April 10...,Let's break down the clues and solve the puzzl...,"Step-by-step solution:\n \n 1. From clue 1, we...",Step-by-step solution:\n \n Step 1: Analyze th...,"Step-by-step solution:\n \n 1. From clue 3, we...",Jamie never misses an opportunity to watch the...,Clues:\n1. The outing to Eastbrook logged 7 fe...,"shooting stars : 2, 9, 16, 23.\ndates : April ...",4,3,stars dates locations


In [5]:
import re
def process_ls(ls):
    if re.search(r"\d+",ls[0]) is None:
        return ls

    else:
        res = []
        for val in ls:
            val = re.sub(r"[^\d+]", '', val)
            res.append(val)
        return res


In [8]:
import re
def process_question(question):
    lines = question.split("\n")
    categories = {}
    clues = []
    actual_question = ""
    ckpt = -1
    flag = -1
    for index, line in enumerate(lines):


        if ckpt == 1 and re.search(r"^\d+\.", line) is None:
            break

        if " : " in line:
            if flag == -1:
                flag = index
            title, ls = line.split(" : ")[0], line.split(" : ")[1].split(", ")
            ls = process_ls(ls)
            categories[title] = ls
        
        if "Clues" in line:
            ckpt = 1
            continue
        
        if ckpt == 1 and re.search(r"\d+\.", line) is not None:
            clues.append(line)
        


    actual_question += "\n".join(lines[:flag])            

    return actual_question, categories, clues


ques, categories, clues = process_question(df["question"].iloc[100])


In [7]:
ind = 12
ques, categories, clues = process_question(df["question"].iloc[ind])


In [13]:
from pprint import pprint
system_prompt = f"""

You are an expert AI assistant for analyzing logic grid puzzles. Your task is to systematically translate puzzle clues into logical expressions.
Analyze each clue and categories input to frame a sequence of logical expressions.

Remember to carefully comprehend each clue step by step and translate them to logical expressions
Retun ONLY a thoughtful sequence of logical expressions.

"""


agent = Agent(system_prompt=system_prompt, agent_name="csp_agent")
user_query= f"""{ques}{"\n".join(clues)}\n"""
for cat in categories:
    user_query += "{cat}: " + ",".join(categories[cat]) + "\n"

print(user_query)

agent_1_response = agent.perform_action(user_query)


The local aerophobe (one with a fear of flying) support group has several members who each need to take a flight. Using only the clues below, match the months to the options from fliers and lucky charms. Remember, as with all grid-based logic puzzles, no option in any category will ever be used more than once.
1. The passenger leaving in February will bring their lucky hat.
2. Christie will leave 1 month after Eloise.
3. The passenger with the wishbone will leave 2 months after Eloise.
4. The aerophobe leaving in May is either the aerophobe with the talisman or the aerophobe with the shamrock.
5. The flier with the coin will leave sometime after the passenger with the shamrock.
6. Of the flier with the coin and the passenger with the talisman, one is Brandi and the other will leave in May.
7. Florence won't leave in March.
{cat}: January,February,March,April,May.
{cat}: Brandi,Christie,Eloise,Florence,Jaime.
{cat}: coin,lucky hat,shamrock,talisman,wishbone.



In [14]:
print(agent_1_response)

Let's denote the months as Jan, Feb, Mar, Apr, May.
Let's denote the fliers as B, C, E, F, J.
Let's denote the lucky charms as coin, hat, sham, tal, wish.

Clue 1: The passenger leaving in February will bring their lucky hat.
Feb = hat

Clue 2: Christie will leave 1 month after Eloise.
C = E + 1 (where +1 means the next month)

Clue 3: The passenger with the wishbone will leave 2 months after Eloise.
wish = E + 2

Clue 4: The aerophobe leaving in May is either the aerophobe with the talisman or the aerophobe with the shamrock.
May = tal OR May = sham

Clue 5: The flier with the coin will leave sometime after the passenger with the shamrock.
coin > sham

Clue 6: Of the flier with the coin and the passenger with the talisman, one is Brandi and the other will leave in May.
(coin = B AND May = tal) OR (tal = B AND May = coin)

Clue 7: Florence won't leave in March.
F != Mar



In [8]:
print(agent_1_response)

Let's denote the categories as follows:
*   Prices: P = {125, 150, 175, 200, 225}
*   Customers: C = {Erica, Francisco, Heidi, Joey, Zachary}
*   Shapes: S = {beach chair, football, golf ball, sports car, train}
*   Flavors: F = {banana, chocolate, peach, raspberry, strawberry}

Now, let's translate the clues into logical expressions:

1.  Zachary's cake isn't strawberry-flavored.
    *   ¬(Zachary = strawberry)

2.  The $150 cake is either Erica's piece or the raspberry-flavored cake.
    *   150 = Erica ∨ 150 = raspberry

3.  Francisco's piece isn't banana-flavored.
    *   ¬(Francisco = banana)

4.  The chocolate-flavored cake costs 25 dollars less than the cake shaped like a sports car.
    *   chocolate = sports car - 25

5.  Of the peach-flavored cake and the cake shaped like a golf ball, one is Erica's and the other is Joey's.
    *   (peach = Erica ∧ golf ball = Joey) ∨ (peach = Joey ∧ golf ball = Erica)

6.  The $125 piece is strawberry-flavored.
    *   125 = strawberry

7.  

In [15]:
user_query = """You are an expert AI assistant for analyzing logical constraints. Analyze the constraints given below and 
map the categories accordingly.
"""
cat_str = """"""
for cat in categories:

    cat_str += f"""{cat}: {", ".join(categories[cat])}\n"""


user_query += cat_str


user_query += agent_1_response + "\n"

return_seq = df.question.iloc[ind].split("Final Answer:")[1]

user_query += return_seq + "\n"

# completion = client.chat.completions.create(
#     model="deepseek-r1-distill-llama-70b",
#     messages=[{
#             "role": "user",
#             "content": user_query

#         }],
#     temperature=0,
#     max_completion_tokens=8192,
#     top_p=1,
#     stream=True,
#     stop=None,
# )

# groq_deepseek_response = ""
# for chunk in completion:
#     text = chunk.choices[0].delta.content or ""
#     groq_deepseek_response += text

# print(groq_deepseek_response)

print(user_query)

You are an expert AI assistant for analyzing logical constraints. Analyze the constraints given below and 
map the categories accordingly.
months: January, February, March, April, May.
fliers: Brandi, Christie, Eloise, Florence, Jaime.
lucky charms: coin, lucky hat, shamrock, talisman, wishbone.
Let's denote the months as Jan, Feb, Mar, Apr, May.
Let's denote the fliers as B, C, E, F, J.
Let's denote the lucky charms as coin, hat, sham, tal, wish.

Clue 1: The passenger leaving in February will bring their lucky hat.
Feb = hat

Clue 2: Christie will leave 1 month after Eloise.
C = E + 1 (where +1 means the next month)

Clue 3: The passenger with the wishbone will leave 2 months after Eloise.
wish = E + 2

Clue 4: The aerophobe leaving in May is either the aerophobe with the talisman or the aerophobe with the shamrock.
May = tal OR May = sham

Clue 5: The flier with the coin will leave sometime after the passenger with the shamrock.
coin > sham

Clue 6: Of the flier with the coin and th

In [10]:
print(df.answer.iloc[ind])

$125 | Francisco | train | strawberry
$150 | Erica | football | peach
$175 | Joey | golf ball | raspberry
$200 | Zachary | beach chair | chocolate
$225 | Heidi | sports car | banana


In [20]:
from typing import List, Union
from pydantic import BaseModel, Field

class ConstraintModel(BaseModel):
    constraints_with_reasoning: List[str]


def get_gemini_response(query=None):
    
    if query is None:
        return query
    
    system_prompt = f"""
                        You are an expert AI assistant for analyzing logic grid puzzles. Your task is to systematically translate puzzle clues into logical exptressions.
                        Analyze each clue and categories input to frame a sequence of logical expressions.

                        Remember to carefully comprehend each clue step by step and translate them to logical expressions
                        Retun ONLY a sequence of logical constraints.

                        """


    agent = Agent(system_prompt=system_prompt, agent_name="csp_agent", json_schema=ConstraintModel)
    #user_query= f"""{ques}\n\n{clues}\n\n{categories}"""

    gemini_response = agent.perform_action(query)

    return gemini_response


In [19]:
def get_groq_response(agent_1_response, question, feedback=None):

    query = """"""

    system_prompt = """You are an expert AI assistant for analyzing logical constraints. Analyze the constraints given below and 
    map the categories accordingly."""
    
    ques, categories, clues = process_question(question)

    cat_str = """"""
    for cat in categories:

        cat_str += f"""{cat}: {", ".join(categories[cat])}\n"""


    system_prompt

    query +=  agent_1_response + "\n" 
    query += "Categories: \n" + cat_str + "\n"


    if feedback is not None:
        if isinstance(feedback, dict):
            for index, key in enumerate(feedback):
                if index == 0:
                    query += f"Feedback: \n {key}: \n" 
                    query += "\n".join(feedback[key]) + "\n"

                else:
                    query += f"{key}: {feedback[key]}\n"

    return_seq = question.split("Final Answer:")[1]

    query += return_seq + "\n"

    completion = client.chat.completions.create(
        model="deepseek-r1-distill-llama-70b",
        messages=[{
                "role": "user",
                "content": query

            },
            {"role": "system", "content": system_prompt}],
        temperature=0,
        max_completion_tokens=5000,
        top_p=1,
        stream=True,
        stop=None,
    )

    groq_deepseek_response = ""
    for chunk in completion:
        text = chunk.choices[0].delta.content or ""
        groq_deepseek_response += text

    return groq_deepseek_response

In [17]:
import json
from tqdm import tqdm
import traceback
def invoke_agentic_flow(df_question):

    groq_response = None
    final_response = ""
    
    ques, categories, clues = process_question(df_question)
    user_query= f"""{ques}\n\n{clues}\n\n{categories}"""
    
    agent_1_response = get_gemini_response(query=user_query)
    print("agent_1_response", agent_1_response)

    try:
        groq_response = get_groq_response(agent_1_response=agent_1_response, question=df_question)
        print("groq_response", groq_response)
            
    except Exception as e:
        return final_response

    final_response = groq_response
    return final_response
    

In [10]:
path = "./agentic_v2_responses"

if not os.path.exists(path):
    os.mkdir(path)

In [ ]:
from tqdm import tqdm
import time
for i,row in tqdm(df.iterrows()):

    if i == 30:
    
        id_ = row["id"]
        question = row["question"]

        print(question)
        reward_, final_response  = invoke_agentic_flow(question)

0it [00:00, ?it/s]

Several passport applications were received at the embassy today, and each needs to be processed. Using only the clues below, match the birthdays to the options from names and careers. Remember, as with all grid-based logic puzzles, no option in any category will ever be used more than once.

birthdays : April 13th, May 18th, June 14th, July 16th.
names : Adrian, Dale, Harry, Lamar.
careers : architect, lawyer, musician, surgeon.

Clues:
1. Harry is the musician.
2. Dale has a birthday 1 month after Harry.
3. Adrian has a birthday 1 month after the surgeon.
4. The architect has a birthday 2 months after the surgeon.

While answering use the following format:
Step-by-step solution:
Your steps showing how you are solving the puzzle
Final Answer:
Fill the following table to show your final answer.
April 13th | correct option from names | correct option from careers
May 18th | correct option from names | correct option from careers
June 14th | correct option from names | correct option fro

30it [00:03,  8.27it/s]


KeyboardInterrupt: 